Table 5. Model and results summary for Opportunity dataset

In [1]:
%pylab inline
import pickle
import glob
import os
import pandas as pd
import numpy as np
import sys

sys.path.insert(0, ".")

import seaborn as sns

sns.set()
sns.set_style("whitegrid")
sns.set_context("paper")


def pd_full(rows=100):
    pd.set_option("display.max_rows", rows)
    pd.set_option("display.max_columns", None)
    pd.set_option("display.width", 2000)
    pd.set_option("display.float_format", "{:20,.3f}".format)
    pd.set_option("display.max_colwidth", None)


pd_full()

Duplicate key in file '/home/ubuntu/.config/matplotlib/matplotlibrc' line #2.
Duplicate key in file '/home/ubuntu/.config/matplotlib/matplotlibrc' line #3.


Populating the interactive namespace from numpy and matplotlib


In [2]:
# Load relevant experiment script
import scripts.run_base_configs_exp as exp
from filternet.training.evalmodel import *

In [3]:
exp.saved_model_glob

'saved_models/base_configs_7*/evalmodel.pth'

In [4]:
# Load saved models produced by script.
model_dirs = [os.path.split(p)[0] for p in glob.glob(exp.saved_model_glob)]
# And also deepconvlstm from a previous run (code for that didn't change)
model_dirs += [
    os.path.split(p)[0]
    for p in glob.glob("saved_models/base_configs_5_deepconvlstm*/evalmodel.pth")
]

In [5]:
df_run_records = []

In [6]:
# Load and summarize reference architectures + deepconvlstm
for i, model_dir in enumerate(model_dirs):
    print(f"{i}/{len(model_dirs)}")

    em = load_eval_model_from_dir(model_dir)

    df = em.trainer.train_state.to_df()
    s = df.loc[em.trainer.cp_iter]

    base_config = em.trainer.base_config
    scale = em.trainer.model_config.get("scale", 1.0)
    if scale != 1.0:
        base_config = f"{base_config}_{scale:.2f}"

    df_run_records.append(
        {
            "model_dir": model_dir,
            "base_config": base_config,
            "loss": s.val_loss,
            "f1w": em.f1,
            "f1m": em.f1_mean,
            "f1w_nn": em.nonull_f1,
            "event_f1": em.event_f1,
            "t_infer": em.infer_time_s_wall,
            "ksamp_per_s": em.infer_samples_per_s / 1000,
            "train_iter_s": df.iter_s_wall.mean(),
            "cp_iter": em.trainer.cp_iter,
            "train_s": em.trainer.cp_iter * df.iter_s_wall.mean(),
        }
    )

0/83
1/83
2/83
3/83
4/83
5/83
6/83
7/83
8/83
9/83
10/83
11/83
12/83
13/83
14/83
15/83
16/83
17/83
18/83
19/83
20/83
21/83
22/83
23/83
24/83
25/83
26/83
27/83
28/83
29/83
30/83
31/83
32/83
33/83
34/83
35/83
36/83
37/83
38/83
39/83
40/83
41/83
42/83
43/83
44/83
45/83
46/83
47/83
48/83
49/83
50/83
51/83
52/83
53/83
54/83
55/83
56/83
57/83
58/83
59/83
60/83
61/83
62/83
63/83
64/83
65/83
66/83
67/83
68/83
69/83
70/83
71/83
72/83
73/83
74/83
75/83
76/83
77/83
78/83
79/83
80/83
81/83
82/83


In [7]:
# Also Load and summarize the ensembles

saved_model_glob2 = "saved_models/ensembles_3_4_folds_?/evalmodel.pth"
model_dirs2 = [os.path.split(p)[0] for p in glob.glob(saved_model_glob2)]
model_dirs2

['saved_models/ensembles_3_4_folds_8',
 'saved_models/ensembles_3_4_folds_1',
 'saved_models/ensembles_3_4_folds_7',
 'saved_models/ensembles_3_4_folds_2',
 'saved_models/ensembles_3_4_folds_3',
 'saved_models/ensembles_3_4_folds_9',
 'saved_models/ensembles_3_4_folds_6',
 'saved_models/ensembles_3_4_folds_0',
 'saved_models/ensembles_3_4_folds_4',
 'saved_models/ensembles_3_4_folds_5']

In [8]:
for i, model_dir in enumerate(model_dirs2):
    print(f"{i}/{len(model_dirs2)}")

    em = load_eval_model_from_dir(model_dir)

    df_run_records.append(
        {
            "model_dir": model_dir,
            "base_config": "4-fold multi_scale_cnn_lstm Ensemble",
            "f1w": em.f1,
            "f1m": em.f1_mean,
            "f1w_nn": em.nonull_f1,
            "event_f1": em.event_f1,
            "t_infer": em.infer_time_s_wall,
            "ksamp_per_s": em.infer_samples_per_s / 1000,
        }
    )

0/10
1/10
2/10
3/10
4/10
5/10
6/10
7/10
8/10
9/10


In [9]:
df_run_records = pd.DataFrame(df_run_records)
df_run_records

,model_dir,base_config,loss,f1w,f1m,f1w_nn,event_f1,t_infer,ksamp_per_s,train_iter_s,cp_iter,train_s
0,saved_models/base_configs_7_multi_scale_cnn_lstm_8,multi_scale_cnn_lstm,0.268,0.929,0.742,0.898,0.846,0.226,"1,047.586",4.998,61.000,304.866
1,saved_models/base_configs_7_base_cnn_8,base_cnn,0.355,0.902,0.642,0.797,0.640,0.174,"1,358.368",1.927,46.000,88.650
2,saved_models/base_configs_7_mscl_p5x_6,multi_scale_cnn_lstm_0.50,0.288,0.911,0.649,0.855,0.774,0.175,"1,354.359",5.130,66.000,338.563
3,saved_models/base_configs_7_multi_scale_cnn_2,multi_scale_cnn,0.304,0.922,0.722,0.889,0.821,0.212,"1,116.419",3.540,29.000,102.666
4,saved_models/base_configs_7_base_lstm_9,base_lstm,0.409,0.893,0.585,0.787,0.581,0.270,877.049,15.541,17.000,264.195
5,saved_models/base_configs_7_base_cnn_3,base_cnn,0.350,0.901,0.642,0.813,0.659,0.182,"1,302.118",1.954,57.000,111.350
6,saved_models/base_configs_7_mscl_p5x_2,multi_scale_cnn_lstm_0.50,0.304,0.912,0.679,0.865,0.808,0.180,"1,310.781",5.272,43.000,226.681
7,saved_models/base_configs_7_cnn_lstm_3,cnn_lstm,0.297,0.927,0.730,0.884,0.826,0.195,"1,215.171",3.969,60.000,238.129
8,saved_models/base_configs_7_multi_scale_cnn_lstm_1,multi_scale_cnn_lstm,0.280,0.923,0.739,0.904,0.812,0.223,"1,062.394",5.106,36.000,183.830
9,saved_models/base_configs_7_base_lstm_2,base_lstm,0.401,0.891,0.600,0.792,0.565,0.273,867.468,15.556,21.000,326.669


In [10]:
df_run_records.loc[df_run_records.base_config == "", "base_config"] = "deepconvlstm"

In [11]:
df_results = df_run_records.groupby("base_config").mean()
df_results["n"] = df_run_records.groupby("base_config").model_dir.count()
df_results

,loss,f1w,f1m,f1w_nn,event_f1,t_infer,ksamp_per_s,train_iter_s,cp_iter,train_s,n
base_config,,,,,,,,,,,
4-fold multi_scale_cnn_lstm Ensemble,nan,0.933,0.755,0.918,0.872,0.780,303.276,nan,nan,nan,10
base_cnn,0.341,0.900,0.638,0.803,0.644,0.177,"1,339.534",1.964,52.111,102.374,9
base_lstm,0.402,0.895,0.595,0.787,0.566,0.274,864.491,15.537,22.444,348.699,9
cnn_lstm,0.296,0.922,0.717,0.883,0.822,0.204,"1,161.332",3.970,49.222,195.494,9
deepconvlstm,0.479,0.868,0.492,0.676,0.514,25.456,9.425,165.961,10.750,"1,784.078",20
multi_scale_cnn,0.288,0.919,0.718,0.891,0.792,0.207,"1,142.500",3.485,49.222,171.377,9
multi_scale_cnn_lstm,0.279,0.928,0.743,0.903,0.842,0.223,"1,060.161",5.061,55.000,278.344,9
multi_scale_cnn_lstm_0.50,0.292,0.921,0.699,0.880,0.815,0.176,"1,345.878",5.211,61.222,318.848,9
multi_scale_cnn_lstm_2.00,0.274,0.927,0.736,0.901,0.841,0.347,681.784,7.000,36.000,251.946,9


In [18]:
df_archs = pd.DataFrame(
    {
        "output_stride_ratio": {
            "base_cnn": 8,
            "multi_scale_cnn": 8,
            "cnn_lstm": 8,
            "multi_scale_cnn_lstm": 8,
            "4-fold multi_scale_cnn_lstm Ensemble": 8,
            "base_lstm": 1,
            "multi_scale_cnn_lstm_0.50": 8,
            "multi_scale_cnn_lstm_2.00": 8,
            "deepconvlstm": 12,
        },
        "cone_size": {
            "base_cnn": 61,
            "multi_scale_cnn": 765,
            "cnn_lstm": 0,
            "multi_scale_cnn_lstm": 0,
            "4-fold multi_scale_cnn_lstm Ensemble": 0,
            "base_lstm": 0,
            "multi_scale_cnn_lstm_0.50": 0,
            "multi_scale_cnn_lstm_2.00": 0,
            "deepconvlstm": 24,
        },
        "modules": {
            "base_cnn": 6,
            "multi_scale_cnn": 11,
            "cnn_lstm": 8,
            "multi_scale_cnn_lstm": 12,
            "4-fold multi_scale_cnn_lstm Ensemble": 12,
            "base_lstm": 3,
            "multi_scale_cnn_lstm_0.50": 12,
            "multi_scale_cnn_lstm_2.00": 12,
            "deepconvlstm": 0,
        },
        "trainable_params": {
            "base_cnn": 209118,
            "multi_scale_cnn": 262338,
            "cnn_lstm": 299818,
            "multi_scale_cnn_lstm": 342838,
            "4-fold multi_scale_cnn_lstm Ensemble": 1371352,
            "base_lstm": 87518,
            "multi_scale_cnn_lstm_0.50": 100936,
            "multi_scale_cnn_lstm_2.00": 1250469,
            "deepconvlstm": 3965778,
        },
    }
)  # see architecture notebook
df_results["output_stride_ratio"] = df_archs["output_stride_ratio"]
df_results["trainable_params_k"] = df_archs.trainable_params / 1000
df_results["trainable_params_k"] = (
    df_results["trainable_params_k"].fillna(0).astype(int)
)

df_results["roi"] = df_archs.cone_size.astype(int)

In [19]:
####
# Table for paper

df_display = df_results.loc[
    [
        "base_lstm",
        "base_cnn",
        "cnn_lstm",
        "multi_scale_cnn",
        "multi_scale_cnn_lstm",
        "4-fold multi_scale_cnn_lstm Ensemble",
        "multi_scale_cnn_lstm_0.50",
        "multi_scale_cnn_lstm_2.00",
        "deepconvlstm",
    ],
    #     :,
    [
        "n",
        "output_stride_ratio",
        "trainable_params_k",
        "roi",
        "f1w",
        "f1m",
        "event_f1",
        "f1w_nn",
        "train_iter_s",
        "ksamp_per_s",  # , 'train_iter_s',
        "cp_iter",
    ],
]
df_display

,n,output_stride_ratio,trainable_params_k,roi,f1w,f1m,event_f1,f1w_nn,train_iter_s,ksamp_per_s,cp_iter
base_config,,,,,,,,,,,
base_lstm,9,1,87,0,0.895,0.595,0.566,0.787,15.537,864.491,22.444
base_cnn,9,8,209,61,0.900,0.638,0.644,0.803,1.964,"1,339.534",52.111
cnn_lstm,9,8,299,0,0.922,0.717,0.822,0.883,3.970,"1,161.332",49.222
multi_scale_cnn,9,8,262,765,0.919,0.718,0.792,0.891,3.485,"1,142.500",49.222
multi_scale_cnn_lstm,9,8,342,0,0.928,0.743,0.842,0.903,5.061,"1,060.161",55.000
4-fold multi_scale_cnn_lstm Ensemble,10,8,1371,0,0.933,0.755,0.872,0.918,nan,303.276,nan
multi_scale_cnn_lstm_0.50,9,8,100,0,0.921,0.699,0.815,0.880,5.211,"1,345.878",61.222
multi_scale_cnn_lstm_2.00,9,8,1250,0,0.927,0.736,0.841,0.901,7.000,681.784,36.000
deepconvlstm,20,12,3965,24,0.868,0.492,0.514,0.676,165.961,9.425,10.750


In [14]:
# Some exploration

In [15]:
df_run_records.groupby("base_config").std()

,loss,f1w,f1m,f1w_nn,event_f1,t_infer,ksamp_per_s,train_iter_s,cp_iter,train_s
base_config,,,,,,,,,,
4-fold multi_scale_cnn_lstm Ensemble,nan,0.003,0.016,0.009,0.010,0.017,6.692,nan,nan,nan
base_cnn,0.011,0.003,0.008,0.008,0.015,0.002,18.311,0.036,11.868,23.667
base_lstm,0.013,0.004,0.021,0.018,0.024,0.002,6.278,0.040,2.920,45.154
cnn_lstm,0.013,0.003,0.017,0.012,0.014,0.007,38.769,0.016,16.084,64.168
deepconvlstm,0.017,0.009,0.037,0.028,0.029,2.813,0.923,1.109,3.626,601.167
multi_scale_cnn,0.012,0.005,0.014,0.010,0.025,0.007,39.843,0.082,17.950,61.488
multi_scale_cnn_lstm,0.011,0.005,0.018,0.010,0.021,0.002,10.319,0.048,18.668,94.757
multi_scale_cnn_lstm_0.50,0.013,0.007,0.029,0.018,0.023,0.003,23.047,0.045,10.121,51.909
multi_scale_cnn_lstm_2.00,0.012,0.003,0.012,0.010,0.006,0.004,8.020,0.014,7.280,50.619


In [16]:
df_run_records.groupby("base_config").min().model_dir.to_dict()

{'4-fold multi_scale_cnn_lstm Ensemble': 'saved_models/ensembles_3_4_folds_0',
 'base_cnn': 'saved_models/base_configs_7_base_cnn_1',
 'base_lstm': 'saved_models/base_configs_7_base_lstm_1',
 'cnn_lstm': 'saved_models/base_configs_7_cnn_lstm_1',
 'deepconvlstm': 'saved_models/base_configs_5_deepconvlstm_0',
 'multi_scale_cnn': 'saved_models/base_configs_7_multi_scale_cnn_1',
 'multi_scale_cnn_lstm': 'saved_models/base_configs_7_multi_scale_cnn_lstm_1',
 'multi_scale_cnn_lstm_0.50': 'saved_models/base_configs_7_mscl_p5x_1',
 'multi_scale_cnn_lstm_2.00': 'saved_models/base_configs_7_mscl_2x_1'}